In [5]:
# Import the needed libraries
import pandas as pd
from Bio import SeqIO, AlignIO
from Bio.Align import AlignInfo
from concurrent.futures import ThreadPoolExecutor
from LabelExtraction import extract_experimental
from FeatureExtractionOperations import calculate_a_acid_composition, calculate_hydrophobicity, calculate_polarity, calculate_mw, calculate_pI

In [2]:
# Define the unaligned dataframe that will have the calculated feature values
unaligned_data = {"ID": [], 
                  "Unaligned Sequence": [], 
                  'A': [], 'R': [], 'N': [], 'D': [],
                  'C': [], 'E': [], 'Q': [], 'G': [],
                  'H': [], 'I': [], 'L': [], 'K': [],
                  'M': [], 'F': [], 'P': [], 'S': [],
                  'T': [], 'W': [], 'Y': [], 'V': [],
                  "Hydrophobicity (Kyte-Doolittle Scale)": [],
                  "Net Charge at pH 7.0 (Neutral)": [],
                  "Net Charge at pH 3.0 (Acidic)": [],
                  "Net Charge at pH 11.0 (Basic)": [],
                  "Isoelectric Point": [],
                  "Molecular Weight": [],
                  "Sequence Length": []} 

for seq_record in SeqIO.parse("sequences.fasta", "fasta"):
    
    unaligned_data["ID"].append(seq_record.id)
    unaligned_data["Unaligned Sequence"].append(str(seq_record.seq))
    
    aa_composition = calculate_a_acid_composition(str(seq_record.seq))
    for amino_acid, percent in aa_composition.items():
        unaligned_data[amino_acid].append(percent)
    
    hydrophobicity_values = calculate_hydrophobicity(str(seq_record.seq))
    unaligned_data["Hydrophobicity (Kyte-Doolittle Scale)"].append(hydrophobicity_values)
    
    charge_7 = calculate_polarity(str(seq_record.seq), 7.0)
    unaligned_data["Net Charge at pH 7.0 (Neutral)"].append(charge_7)

    charge_3 = calculate_polarity(str(seq_record.seq), 3.0)
    unaligned_data["Net Charge at pH 3.0 (Acidic)"].append(charge_3)

    charge_11 = calculate_polarity(str(seq_record.seq), 11.0)
    unaligned_data["Net Charge at pH 11.0 (Basic)"].append(charge_11)
    
    isolectric_values = calculate_pI(str(seq_record.seq))
    unaligned_data["Isoelectric Point"].append(isolectric_values)
    
    mw_values = calculate_mw(str(seq_record.seq))
    unaligned_data["Molecular Weight"].append(mw_values)
            
    sequence_length = len(seq_record.seq)
    unaligned_data["Sequence Length"].append(sequence_length)    

In [3]:
unaligned_df = pd.DataFrame(unaligned_data)
unaligned_df.head()

,ID,Unaligned Sequence,A,R,N,D,C,E,Q,G,...,W,Y,V,Hydrophobicity (Kyte-Doolittle Scale),Net Charge at pH 7.0 (Neutral),Net Charge at pH 3.0 (Acidic),Net Charge at pH 11.0 (Basic),Isoelectric Point,Molecular Weight,Sequence Length
0,2VO6_A,ESVKEFLAKAKEDFLKKWENPAQNTAHLQFERIKTLGTGSFGRVML...,0.060060,0.045045,0.045045,0.051051,0.006006,0.078078,0.036036,0.060060,...,0.018018,0.042042,0.054054,"[-0.33333333333333337, 0.2555555555555556, -0....",5.643805,55.488643,-42.070511,8.866353,38862.3721,333
1,2VO6_I,TTYADFIASGRTGRRNAIHD,0.150000,0.150000,0.050000,0.100000,0.000000,0.000000,0.000000,0.100000,...,0.000000,0.050000,0.000000,"[0.43333333333333335, 0.46666666666666656, 0.0...",0.489764,4.807728,-1.181742,8.435640,2222.3766,20
2,3DNK_A,GGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDG...,0.028708,0.028708,0.047847,0.047847,0.019139,0.071770,0.043062,0.043062,...,0.019139,0.043062,0.110048,"[0.9777777777777779, 0.8444444444444444, 0.455...",-0.730126,29.441514,-31.050926,6.658938,23694.5096,209
3,3DNK_B,GGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDG...,0.028571,0.028571,0.047619,0.047619,0.019048,0.071429,0.042857,0.042857,...,0.019048,0.042857,0.109524,"[0.9777777777777779, 0.8444444444444444, 0.455...",-0.730126,29.441514,-31.050926,6.658938,23791.6248,210
4,3GLT_A,GKLSLQDVAELIRARACQRVVVMVGAGISTPSGIPDFRSPGSGLYS...,0.062044,0.069343,0.021898,0.054745,0.018248,0.058394,0.032847,0.072993,...,0.007299,0.025547,0.098540,"[0.16666666666666666, -0.17777777777777784, 0....",-3.632580,32.997043,-25.313742,5.998298,30460.8986,274


In [4]:
# Read the alignment sequences
alignment = AlignIO.read("aligned_sequences.fasta", "fasta")

# Calculate consensus
consensus = AlignInfo.SummaryInfo(alignment).dumb_consensus()

# Calculate Conservation Score
start = 0
end = alignment.get_alignment_length()
e_freq_table = {char: 0.05 for char in "ACDEFGHIKLMNPQRSTVWY"}
conservation_score = AlignInfo.SummaryInfo(alignment).information_content(start, end, e_freq_table=e_freq_table, chars_to_ignore=["-"])

# Initialize variables to store gap statistics
alignment_length = end
num_sequences = len(alignment)
gap_count_per_position = [0] * alignment_length

# Count the number of gaps at each position
for seq_record in alignment:
    for i, residue in enumerate(str(seq_record.seq)):
        if residue == "-":
            gap_count_per_position[i] += 1
            
# Calculate the percentage of gaps at each position
perc_gap_per_position = [count / num_sequences * 100 for count in gap_count_per_position]

# Calculate total number of gaps
total_gaps = sum(gap_count_per_position)

# Calculate average gap length
all_gaps = []
for seq_record in alignment:
    sequence = str(seq_record.seq)
    gaps = [gap for gap in sequence.split('-') if gap]
    gaps_length = [len(gap) for gap in gaps]
    all_gaps.extend(gaps_length)

average_gap_length = sum(all_gaps) / len(all_gaps) if all_gaps else 0

# Define the aligned dataframe that will have the calculated feature values
aligned_data = {"ID": [], 
                "Aligned Sequence": [],
                "Consensus Sequence": [str(consensus)] * num_sequences, 
                "Conservation Scores": [conservation_score] * num_sequences,
                "Percentage of Gaps Per Position": [perc_gap_per_position] * num_sequences,
                "Total Gaps in Alignment": [total_gaps] * num_sequences,
                "Average Gap Length": [average_gap_length] * num_sequences,
                "Sequence Length": [],
                "Gap Count": [],
                "Percentage Gaps": [],
                "Mutations from Consensus": []}

for seq_record in alignment:
    
    aligned_data["ID"].append(seq_record.id)
    aligned_data["Aligned Sequence"].append(str(seq_record.seq))
    
    sequence = str(seq_record.seq)
    len_sequence = len(sequence)
    gap_count = sequence.count('-')
    perc_gaps = (gap_count / len_sequence) * 100
    mutations_from_consensus = sum(c1 != c2 for c1, c2 in zip(sequence, consensus))

    aligned_data["Sequence Length"].append(len_sequence)
    aligned_data["Gap Count"].append(gap_count)
    aligned_data["Percentage Gaps"].append(perc_gaps)
    aligned_data["Mutations from Consensus"].append(mutations_from_consensus)

In [5]:
aligned_df = pd.DataFrame(aligned_data)
aligned_df.head()

,ID,Aligned Sequence,Consensus Sequence,Conservation Scores,Percentage of Gaps Per Position,Total Gaps in Alignment,Average Gap Length,Sequence Length,Gap Count,Percentage Gaps,Mutations from Consensus
0,2VO6_A,----------------------------------------------...,PEXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,6188.401365,"[99.96420581655481, 99.96420581655481, 99.9015...",53679017,3.557247,5024,4691,93.371815,5024
1,2VO6_I,----------------------------------------------...,PEXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,6188.401365,"[99.96420581655481, 99.96420581655481, 99.9015...",53679017,3.557247,5024,5004,99.601911,5024
2,3DNK_A,----------------------------------------------...,PEXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,6188.401365,"[99.96420581655481, 99.96420581655481, 99.9015...",53679017,3.557247,5024,4815,95.839968,5024
3,3DNK_B,----------------------------------------------...,PEXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,6188.401365,"[99.96420581655481, 99.96420581655481, 99.9015...",53679017,3.557247,5024,4814,95.820064,5024
4,3GLT_A,----------------------------------------------...,PEXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,6188.401365,"[99.96420581655481, 99.96420581655481, 99.9015...",53679017,3.557247,5024,4750,94.546178,5024


In [6]:
merged_df = pd.merge(unaligned_df, aligned_df, on="ID")
merged_df.head()

,ID,Unaligned Sequence,A,R,N,D,C,E,Q,G,...,Aligned Sequence,Consensus Sequence,Conservation Scores,Percentage of Gaps Per Position,Total Gaps in Alignment,Average Gap Length,Sequence Length_y,Gap Count,Percentage Gaps,Mutations from Consensus
0,2VO6_A,ESVKEFLAKAKEDFLKKWENPAQNTAHLQFERIKTLGTGSFGRVML...,0.060060,0.045045,0.045045,0.051051,0.006006,0.078078,0.036036,0.060060,...,----------------------------------------------...,PEXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,6188.401365,"[99.96420581655481, 99.96420581655481, 99.9015...",53679017,3.557247,5024,4691,93.371815,5024
1,2VO6_I,TTYADFIASGRTGRRNAIHD,0.150000,0.150000,0.050000,0.100000,0.000000,0.000000,0.000000,0.100000,...,----------------------------------------------...,PEXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,6188.401365,"[99.96420581655481, 99.96420581655481, 99.9015...",53679017,3.557247,5024,5004,99.601911,5024
2,3DNK_A,GGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDG...,0.028708,0.028708,0.047847,0.047847,0.019139,0.071770,0.043062,0.043062,...,----------------------------------------------...,PEXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,6188.401365,"[99.96420581655481, 99.96420581655481, 99.9015...",53679017,3.557247,5024,4815,95.839968,5024
3,3DNK_B,GGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDG...,0.028571,0.028571,0.047619,0.047619,0.019048,0.071429,0.042857,0.042857,...,----------------------------------------------...,PEXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,6188.401365,"[99.96420581655481, 99.96420581655481, 99.9015...",53679017,3.557247,5024,4814,95.820064,5024
4,3GLT_A,GKLSLQDVAELIRARACQRVVVMVGAGISTPSGIPDFRSPGSGLYS...,0.062044,0.069343,0.021898,0.054745,0.018248,0.058394,0.032847,0.072993,...,----------------------------------------------...,PEXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,6188.401365,"[99.96420581655481, 99.96420581655481, 99.9015...",53679017,3.557247,5024,4750,94.546178,5024


In [6]:
# Define the label dataframe that will be the prediction outputs
label_data = {"ID": [],
              "Experimental": [],
              "Resolution": [],
              "R Value": [],
              "R Free": []}

def extract_data(seq_record):
    try:
        r_value = None
        r_free = None
        seq_id = seq_record.id
        base_pdb_id = seq_id.split("_")[0]
        pdb_file = f"PDBData/{base_pdb_id}.pdb"
        experimental, res = extract_experimental(pdb_file)
        
        with open(pdb_file, "r") as f:
            for line in f:
                try:
                    if "REMARK   3   R VALUE            (WORKING SET) :" in line:
                        r_value = float(line.split()[-1])
                    elif "REMARK   3   FREE R VALUE                     :" in line:
                        r_free  = float(line.split()[-1])
                except ValueError:
                    pass
                    
        return seq_id, experimental, res, r_value, r_free
    except ValueError:
        return seq_id, None, None, None, None

with (ThreadPoolExecutor() as executor):
    for seq_id, experimental, res, r_value, r_free in executor.map(extract_data, SeqIO.parse("sequences.fasta", "fasta")):
        label_data["ID"].append(seq_id)
        label_data["Experimental"].append(experimental)
        label_data["Resolution"].append(res)
        label_data["R Value"].append(r_value)
        label_data["R Free"].append(r_free)

In [7]:
label_df = pd.DataFrame(label_data)
label_df.head()

,ID,Experimental,Resolution,R Value,R Free
0,2VO6_A,x-ray diffraction,1.97,0.172,0.229
1,2VO6_I,x-ray diffraction,1.97,0.172,0.229
2,3DNK_A,x-ray diffraction,2.84,0.240,0.320
3,3DNK_B,x-ray diffraction,2.84,0.240,0.320
4,3GLT_A,x-ray diffraction,2.10,0.195,0.228


In [8]:
# Print statement for debugging purposes
print(label_df[label_df["ID"].str.contains("1KJL")])

         ID       Experimental  Resolution  R Value  R Free
880  1KJL_A  x-ray diffraction         1.4    0.186   0.217
